In [1]:
from tests.auth_utils import get_toolkit_client

In [21]:
from datetime import datetime

from cognite.client.data_classes import TimeSeriesWrite
from cognite.client.data_classes.data_modeling import NodeId, SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteTimeSeriesApply
from cognite.client.exceptions import CogniteException

In [3]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [4]:
SPACE = "sp_upgrade"

In [5]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Cascading Delete

In [13]:
ts = TimeSeriesWrite(
    external_id="ts_case2_cascading_delete",
    name="Case 2 Cascading Delete",
    is_step=False,
    is_string=False,
)

In [14]:
# Ensure the time series does not exist before running the test
client.time_series.delete(external_id=ts.external_id, ignore_unknown_ids=True)

In [15]:
created = client.time_series.create(ts)
created

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
is_string,False
metadata,{}
is_step,False
security_categories,[]
id,3635063918976811
created_time,2025-07-10 05:59:12.763000
last_updated_time,2025-07-10 05:59:12.763000


In [16]:
datapoints = [
    {"timestamp": datetime(2020, 1, 1, 0, 0, 0), "value": 1.0},
    {"timestamp": datetime(2020, 1, 2, 0, 0, 0), "value": 2.0},
    {"timestamp": datetime(2020, 1, 3, 0, 0, 0), "value": 3.0},
]

In [17]:
client.time_series.data.insert(datapoints, external_id=ts.external_id)

In [18]:
updated = client.time_series.set_pending_ids(NodeId(SPACE, ts.external_id), external_id=ts.external_id)
updated

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
is_string,False
metadata,{}
is_step,False
security_categories,[]
id,3635063918976811
created_time,2025-07-10 05:59:12.763000
last_updated_time,2025-07-10 05:59:18.524000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [19]:
cognite_ts = CogniteTimeSeriesApply(
    space=SPACE,
    external_id=ts.external_id,
    is_step=ts.is_step,
    time_series_type="string" if ts.is_string else "numeric",
    name=ts.name,
)

In [20]:
created_dm = client.data_modeling.instances.apply(cognite_ts).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case2_cascading_delete,1,True,2025-07-10 05:59:32.286,2025-07-10 05:59:32.286


In [22]:
try:
    client.time_series.delete(external_id=ts.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print("\033[92mTimeSeries deleted successfully\033[0m")

Error 400 Time series with instance ids must be deleted through data modeling


In [13]:
deleted = client.data_modeling.instances.delete(cognite_ts.as_id()).nodes
deleted

[NodeId(space='sp_upgrade', external_id='ts_case2_cascading_delete')]

In [14]:
client.time_series.retrieve(external_id=ts.external_id)